In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import numpy as np

In [56]:
# url = "https://www.portalinmobiliario.com/arriendo/departamento/las-condes-metropolitana"
url = "https://www.portalinmobiliario.com/arriendo/departamento/las-condes-metropolitana/_PriceRange_400000CLP-410000CLP"

In [57]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = requests.get(url=url, headers=headers)

soup = BeautifulSoup(req.content, "html.parser")

In [58]:
# all houses in page
dataset = soup.find_all("div", {"class": "rowItem item highlighted item--stack new"}) 
len(dataset)

50

In [59]:
# helper function
def find(string, character):
    """Return all required characters' positions"""
    return [i for i, letter in enumerate(string) if letter == character]

In [60]:
infoset = []
_continue = True
page = 1



while _continue == True:
    print("Página: ", page)
    for apartment in dataset:
        d = {}
        try:
            price = apartment.find("span", {"class": "price__fraction"}).text.strip()
        except:
            price = None

        if price != None:
            price = price.replace('.','')
            d['price'] = int(price)
            
            try:
                d['currency'] = apartment.find("span", {"class": "price__symbol"}).text.strip()
            except:
                d['currency'] = None
            
            try:
                atributes = apartment.find("div", {"class": "item__attrs"}).text.strip()
                indexes = find(atributes,'|')

                # Area
                m2 = atributes[0:indexes[0]].strip()
                d['m2'] = float(m2[:-10])

                # Rooms
                dorms = atributes[indexes[0]+1:indexes[1]].strip()
                if 'dormitorios' in dorms:
                    d['dorms'] = int(dorms[0:-12])
                else:
                    d['dorms'] = int(dorms[0:-11])

                # Bathrooms
                baths = atributes[indexes[1]+1:].strip()
                if 'baños' in baths:
                    d['baths'] = int(baths[0:-6])
                else:
                    d['baths'] = int(baths[0:-5])

            except:
                d['atributes'] = None
                d['m2'] = None
                d['dorms'] = None
                d['baths'] = None

            try:
                link_html = apartment.find("a", {"class": "item__info-link"})
                d['link'] = link_html.get('href')
            except:
                d['link'] = None

            infoset.append(d)

    try:
        next_nav = soup.find("li", {"class": "andes-pagination__button andes-pagination__button--next"})
        next_nav = next_nav.find("a", {"class": "andes-pagination__link prefetch"}) # one more level deep
        next_url = next_nav.get('href')
        req = requests.get( url=next_url, headers=headers )
        time.sleep(1)
        soup = BeautifulSoup(req.content, "html.parser")
        dataset = soup.find_all("div", {"class": "rowItem item highlighted item--stack new"})
        page += 1
    except:
        _continue = False
        print("Terminado")

    

Página:  1
Página:  2
Página:  3
Página:  4
Página:  5
Terminado


In [61]:
# transformar la lista a dataFrame
houses_df = pd.DataFrame(infoset)

In [62]:
houses_df

,price,currency,m2,dorms,baths,link
0,400000,$,37.0,1,1,https://www.portalinmobiliario.com/arriendo/de...
1,410000,$,39.0,1,1,https://www.portalinmobiliario.com/MLC-5347717...
2,410000,$,42.0,1,1,https://www.portalinmobiliario.com/MLC-5350951...
3,400000,$,40.0,1,1,https://www.portalinmobiliario.com/arriendo/de...
4,400000,$,45.0,1,1,https://www.portalinmobiliario.com/arriendo/de...
...,...,...,...,...,...,...
201,400000,$,30.0,1,1,https://www.portalinmobiliario.com/MLC-5351048...
202,14,UF,40.0,1,1,https://www.portalinmobiliario.com/arriendo/de...
203,400000,$,40.0,1,1,https://www.portalinmobiliario.com/arriendo/de...
204,410000,$,38.0,1,1,https://www.portalinmobiliario.com/arriendo/de...
